In [1]:
import os
import cv2
import numpy as np

from utils.io import scan2df
from utils import endoscopy

In [2]:
# for each video find center, radius
# save to configs/cholec80_transforms.yml
folder = '/media/martin/Samsung_T5/data/endoscopic_data/cholec80/videos'

df = scan2df(folder, postfix='.mp4')

In [3]:
for idx, row in df.iterrows():
    ib = endoscopy.ImageBuffer(buffer_size=50)

    vc = cv2.VideoCapture(os.path.join(row.folder, row.file))

    print('Processing file: {}'.format(row.file))

    steady_count = 0
    prev_center, prev_radius = np.array([0, 0], dtype=np.int), None
    stable_steady_count = 20

    # square check
    zoom_count = 0
    stable_zoom_count = 100

    center_found = False
    zoom = False

    while vc.isOpened():

        _, img = vc.read()
        if img is None:
            break

        img = img[5:-5, 5:-5]
        ib.appendBuffer(img)

        avg_binary = ib.binaryAvg(th=20)

        center, radius = endoscopy.ransacBoundaryCircle(avg_binary, th=10, fit='numeric', n_pts=100, n_iter=10)
        top_left, shape = endoscopy.boundaryRectangle(avg_binary, th=200)

        # check for circle fit
        if radius is not None:
            # find max inner rectangle
            inner_top_left, inner_shape = endoscopy.maxRectangleInCircle(img.shape, center, radius)
            inner_top_left, inner_shape = inner_top_left.astype(np.int), tuple(map(int, inner_shape))
    
            center, radius = center.astype(np.int), int(radius)
            top_left, shape = top_left.astype(np.int), tuple(map(int, shape))

            if steady_count == 0:
                prev_center, prev_radius = center, radius
                steady_count += 1
            else:
                if np.isclose(prev_center, center, atol=1.).all():
                    prev_center, prev_radius = center, radius
                    steady_count += 1

                    if steady_count >= stable_steady_count:
                        center_found = True
                        break
                else:
                    prev_center, prev_radius = np.array([0, 0], dtype=np.int), None
                    steady_count = 0

            cv2.circle(img, (center[1], center[0]), radius, (0, 255, 255))
            cv2.circle(img, (center[1], center[0]), 2, (0, 255, 255))
            cv2.rectangle(img, (top_left[1], top_left[0]), (top_left[1]+shape[1], top_left[0]+shape[0]), (255, 0, 255))
            cv2.rectangle(
                img, 
                (inner_top_left[1], inner_top_left[0]), 
                (inner_top_left[1]+inner_shape[1], inner_top_left[0]+inner_shape[0]), 
                (255, 255, 0)
            )

        # check for zoom
        zoomed, confidence = endoscopy.isZoomed(avg_binary, th=0.98)
        if zoomed:
            zoom_count += 1
        else:
            zoom_count = 0
        if zoom_count > stable_zoom_count:
            zoom = True
            break

        cv2.imshow('avg_binary', avg_binary)
        cv2.imshow('img', img)
        cv2.waitKey(1)
        
        print('\rSteady count: {}, zoom count: {}'.format(steady_count, zoom_count), end='')

    vc.release() 
    print('\nCenter found: {}, zoom: {}\n'.format(center_found, zoom))

    if radius is not None:
        cv2.circle(img, (center[1], center[0]), radius, (0, 255, 255))
        cv2.circle(img, (center[1], center[0]), 2, (0, 255, 255))
        cv2.rectangle(img, (top_left[1], top_left[0]), (top_left[1]+shape[1], top_left[0]+shape[0]), (255, 0, 255))
        cv2.rectangle(
            img, 
            (inner_top_left[1], inner_top_left[0]), 
            (inner_top_left[1]+inner_shape[1], inner_top_left[0]+inner_shape[0]), 
            (255, 255, 0)
        )

    cv2.imshow('avg_binary', avg_binary)
    cv2.imshow('img', img)
    cv2.waitKey()


Processing file: video36.mp4
Steady count: 19, zoom count: 0
Center found: True, zoom: False

